# Препроцессинг и тестирование моделей

В этом ноутбуке представлен препроцессинг тестового подкорпуса и обучающей выборки из 1000 примерво, а также сравнивается работа различных моделей.

In [1]:
import gensim
from gensim.models import word2vec
import logging
import openpyxl
import pandas as pd
from pprint import pprint
import pymystem3
from pymystem3 import Mystem
import re
import warnings
warnings.filterwarnings('ignore')

## Обработка выборки 1000 употреблений *то есть* в НКРЯ

In [3]:
## Работа с таблицей по векам

wb = openpyxl.load_workbook('/Users/annaaksenova/Desktop/Работа к Кувшинской/То есть выборка по векам.xlsx')
sheet = wb['21'] # Имя листа

In [4]:
m = Mystem()

In [5]:
v1 = []
# Собираем примеры из таблицы
for v in sheet.iter_rows(min_row=2, min_col=21, max_col=21, max_row=1001, values_only=True):
    if v[0] == None:
        v1.append('')
    else:
        v1.append(v[0].replace(u'\xa0', ' ').strip())
v2 = []
for v in sheet.iter_rows(min_row=2, min_col=22, max_col=22, max_row=1001, values_only=True):
    if v[0] == None:
        v2.append('')
    else:
        item = v[0].replace(u'\xa0', ' ')
        item = item.strip()
        item = item.replace('  ', '')
        item = re.sub('\[.+\]', '', item)
        item = re.sub(r'\b[Тт]о есть', r'тоесть', item) # \b убирает случаи что есть
        item = re.sub(r'\b[Аа] именно', r'аименно', item)
        item = re.sub(r'\b[Тт]ак как', r'таккак', item)
        item = re.sub(r'\b[Пп]отому что', r'потомучто', item)
        v2.append(item)
values = [v1[i]+v2[i] for i in range(1000)]
print(values[:10])

['Я засунула пластинку и блюдо в один пакет, положила под сиденье и ― надо знать наши самолёты советских времён, лишённые кондиционеров, ― в конце пути в Ереван достала из-под сиденья пластинку, принявшую форму таза, тоесть блюда.', 'Само название места происходит от того, что здесь врачевали «косимых» ― тоесть увечных.', 'Я даже не могу найти концов, тоесть понять, кто же мне, собственно, должен платить.', 'Никаких провокаций ― известные имена-брэнды.  Расчёт на то, что, даже если зритель не шибко приобщён к балету и не знает, кто такая Аньес Летестю, он непременно отреагирует на Opera National de Paris.  Или хотя бы на «Лебединое озеро». Или хотя бы на Paris.  Плюс гарантия качества: звёзды подобраны заслуженные, но при этом, что важно, действующие: спешно доставленные на пароходе прямо из Парижа, поднимешь крышку ― пар!тоесть готовые доказывать свой статус каждым спектаклем.', '«Чтобы не быть выдавленными с рынка, ― говорит академик Алексей Макаров, ― необходимы жесткий расчет эконо

In [6]:
# Парсим предложения
def parser(string):
    punctuation = (list(""",;:”$%^&*№()_—=+|[]{}\"/<>`~±§«»°1234567890""")
               + ['- ', ' -', ' - ', '― ', ' ―', ' ― ', " '", "' "])
    string = string.lower()
    for ch in list('.…!?'): # Разбиваем примеры на предложения
        string = string.replace(ch, '\n')
    for char in punctuation:
        string = string.replace(char, ' ')
    lemmas = m.lemmatize(string)
    parsed = ''.join(lemmas)
    parsed = parsed.replace('тоедать', 'тоесть')
    return parsed

In [7]:
data = [parser(example) for example in values]

In [8]:
# Записываем в файл лемматизированные предложения
with open('21_century.txt', 'w', encoding='utf-8') as fw:
    for i in data:
        fw.write(i)

# Препроцессинг корпуса

### Нужно:
1. Соединить слова *то есть, а именно, потому что, так как*
2. Убрать омонимию *если, то; не, а*

Загружаем данные корпуса

In [9]:
import pickle
from pprint import pprint

In [10]:
with open('/Volumes/Анна/Corpora/stable/disamb/corpora/lines_1m_to_3m.pickle', 'rb') as f:
    data_13 = pickle.load(f)

In [11]:
with open('/Volumes/Анна/Corpora/stable/disamb/corpora/lines_3000000.pickle', 'rb') as f:
    data_3 = pickle.load(f)

In [12]:
BigData = data_13 + data_3

Обрабатываем корпус

In [13]:
# Видим точку не в т.к., т.д. или т.е. - новое предложение
text = []  # Список предложений
sentence = []  # Обновляемый список слов в предложении
k = -1  # Счетчик позиции. Вот если у нас "т" из т.е., мы можем не обрабатывать следующие 3 символа
for i in range(len(BigData)-2):
    if i <= k:  # Если счетчик больше, то мы уже учли это слово
        continue
    elif len(BigData[i]) != 7:
        continue
    else:
        k = -1
        if not (BigData[i][0] == '!' or BigData[i][0] == '?' or BigData[i][0] == '.' or BigData[i][0] == '…') :
            if ((BigData[i][0] == 'то' or BigData[i][0] == 'То') 
                and BigData[i][6] == 'предик' and BigData[i+1][0] == 'есть'):
                sentence.append('тоесть')
                k = i + 1  # Не смотрим на следующее "есть"
            elif (BigData[i][0] == 'так' or BigData[i][0] == 'Так') and BigData[i+1][0] == 'как':
                sentence.append('таккак')
                k = i + 1
            elif (BigData[i][0] == 'потому' or BigData[i][0] == 'Потому') and BigData[i+1][0] == 'что':
                sentence.append('потомучто')
                k = i + 1
            elif (BigData[i][0] == 'в' or BigData[i][0] == 'В') and BigData[i+1][0] == 'смысле':
                sentence.append('всмысле')  # их всего 156
                k = i + 1
            elif (BigData[i][0] == 'А' or BigData[i][0] == 'А') and BigData[i+1][0] == 'именно':
                sentence.append('аименно')
                k = i + 1
            elif ((BigData[i][0] == 'т' or BigData[i][0] == 'Т') 
                  and BigData[i+1][0] == '.' and BigData[i+2][0] == 'е'):
                sentence.append('тоесть')
                k = i + 3 
            elif (BigData[i][0] == 'т' or BigData[i][0] == 'Т') and BigData[i+1][0] == '.' and BigData[i+2][0] == 'к':
                sentence.append('таккак')
                k = i + 3
            elif BigData[i][6] == 'PUNC':  # Если не !.?... но знак препинания, не добавляем
                continue
            elif (BigData[i][0] == '(' or BigData[i][0] == ')'):
                continue
            else:
                sentence.append(BigData[i][2])
        elif not (BigData[i+1][0] == 'п' or BigData[i+1][0] == 'д'):  # Если у нас точка, и она не входит в "т.п." и "т.д."
            text.append(sentence)
            sentence = []

In [14]:
len(text)

326833

In [15]:
with open('lems.txt', 'w') as fw:
    for i in text:
        fw.write(' '.join(i).lower())
        fw.write('/n')

# Word2Vec CBOW

Перейдем к рассмотрению моделей. Загрузим файл со всеми примерами употребления *то есть*, что у нас есть

In [16]:
with open("/Volumes/Анна/corp_table.txt", "w") as fw:
    for i in text:
        fw.write(' '.join(i).lower())
        fw.write('/n')
    for i in data:
        fw.write(' '.join(i))

In [17]:
f = '/Volumes/Анна/corp_table.txt'
data = gensim.models.word2vec.LineSentence(f)

In [18]:
model_cbow_full = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50)

In [19]:
model_cbow_full.init_sims(replace=True)
model_path = "to_cbow.bin"

print("Saving model...")
model_cbow_full.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [20]:
# модель относительно хорошо справилась с выделением ближайших соседей
pprint(model_cbow_full.wv.most_similar("тоесть", topn=10))

[('но', 0.23746706545352936),
 ('таккак', 0.2237522304058075),
 ('и', 0.22348180413246155),
 ('вот', 0.22122493386268616),
 ('же', 0.217709481716156),
 ('для', 0.216770738363266),
 ('весь', 0.21245142817497253),
 ('действие', 0.2099384367465973),
 ('а', 0.20862986147403717),
 ('быть', 0.20564895868301392)]


# Word2Vec Skip-gram

Обучим модель на тех же данных

In [21]:
data = gensim.models.word2vec.LineSentence(f)

In [22]:
model_to = gensim.models.Word2Vec(data, size=300, window=5, min_count=5, iter=50, sg=1)

In [23]:
model_to.init_sims(replace=True)
model_path = "to_sg.bin"  # сохраняем модель

print("Saving model...")
model_to.wv.save_word2vec_format(model_path, binary=True)

Saving model...


In [24]:
# как мы видим, ближайшие соседи не очень осознанные
pprint(model_to.wv.most_similar("тоесть", topn=10))

[('ароматно', 0.3942897915840149),
 ('оранжевый/nи', 0.34115463495254517),
 ('охранение', 0.3324703574180603),
 ('цивилизованно', 0.31085678935050964),
 ('передел', 0.303924024105072),
 ('выключатель', 0.30314773321151733),
 ('переложение', 0.29808560013771057),
 ('+5', 0.2975841462612152),
 ('конечно/nлихачева', 0.2955629825592041),
 ('600.', 0.29418084025382996)]


# GloVe

Перейдем к модели на основе матриц совместной встречаемости

In [25]:
with open(f) as fh:
    lines = fh.readlines()
lines = [line.split() for line in lines]

In [26]:
from glove import Corpus, Glove

corpus = Corpus() 

corpus.fit(lines, window=10)  # Окно 10
# Стандартные гиперпараметры для градиентного спуска
glove = Glove(no_components=5, learning_rate=0.05)
 
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [27]:
# Модель не нашла осознанных ближайших соседей
pprint(glove.most_similar('тоесть', 10))

[('–', 0.999085089934544),
 ('проблема', 0.9988588287344782),
 ('возникать', 0.9983956958236553),
 ('честно/n/nи', 0.9977361896826833),
 ('звучать', 0.9971758819368663),
 ('иной', 0.9961107612820005),
 ('отношение', 0.9959756587319284),
 ('программист', 0.9959280364322133),
 ('судить', 0.9958511685208293)]


# AdaGram

Рассмотрим модель, выделяющую нескольк значений одного слова.

In [30]:
! adagram-train /Volumes/Анна/corp_table.txt out1.pkl

[INFO] 2020-05-27 08:34:24,704 Building dictionary...
[INFO] 2020-05-27 08:39:24,047 Done! 21007 words.
[INFO] 2020-05-27 08:44:32,820 1.89% -4.9977 0.0245 1.1/2.0 0.21 kwords/sec
[INFO] 2020-05-27 08:44:35,156 3.79% -3.9994 0.0241 1.0/2.0 27.39 kwords/sec
[INFO] 2020-05-27 08:44:37,600 5.68% -3.6704 0.0236 1.0/2.0 26.19 kwords/sec
[INFO] 2020-05-27 08:44:40,029 7.58% -3.5075 0.0231 1.0/2.0 26.35 kwords/sec
[INFO] 2020-05-27 08:44:42,434 9.47% -3.4105 0.0226 1.0/2.0 26.62 kwords/sec
[INFO] 2020-05-27 08:44:44,910 11.37% -3.3464 0.0222 1.0/2.0 25.84 kwords/sec
[INFO] 2020-05-27 08:44:47,263 13.26% -3.3009 0.0217 1.0/2.0 27.20 kwords/sec
[INFO] 2020-05-27 08:44:49,665 15.16% -3.2670 0.0212 1.0/2.0 26.64 kwords/sec
[INFO] 2020-05-27 08:44:52,029 17.05% -3.2408 0.0207 1.0/2.0 27.07 kwords/sec
[INFO] 2020-05-27 08:44:54,346 18.95% -3.2200 0.0203 1.0/2.0 27.62 kwords/sec
[INFO] 2020-05-27 08:44:56,761 20.84% -3.2031 0.0198 1.0/2.0 26.50 kwords/sec
[INFO] 2020-05-27 08:44:59,064 22.74% -3.189

In [31]:
import adagram
vm = adagram.VectorModel.load('out1.pkl')

In [32]:
vm.word_sense_probs('тоесть')

[(0, 0.9998891475446181)]

In [33]:
# В этом значении соседи не отражают реальность
vm.sense_neighbors('тоесть', 1, 10)

[('дорога/nа', 0, 0.3847712),
 ('отделать', 0, 0.37543383),
 ('депрессия', 0, 0.370262),
 ('вернуться/nно', 0, 0.36666045),
 ('лавра', 0, 0.33810607),
 ('высококлассный', 0, 0.33423528),
 ('ядро', 0, 0.32427284),
 ('наговорить', 0, 0.31927833),
 ('как-будто', 0, 0.3191208),
 ('зарекаться', 0, 0.31838065)]

In [34]:
# В этом значении есть слова где, всякий, кажый, наверное, сопоставимые с то есть
vm.sense_neighbors('тоесть', 0, 10)

[('принцип', 0, 0.98693883),
 ('работать', 0, 0.98508406),
 ('сс', 0, 0.98505765),
 ('где', 0, 0.9844817),
 ('основа', 0, 0.98444),
 ('точка', 0, 0.98418367),
 ('каждый', 0, 0.98337305),
 ('всякий', 0, 0.9824113),
 ('наверное', 0, 0.98168135),
 ('жить', 0, 0.9815976)]

Построим визуализацию отношений соседей *то есть*

In [36]:
from vec2graph import visualize
visualize('./', model_cbow_full.wv, 'тоесть', depth=0, topn=10, threshold=0.3, edge=1, sep=True)

Visualizations written to ./


['тоесть']